# Advent of Code 2020 Day 16
[link](https://adventofcode.com/2020/day/16)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#dependencies" data-toc-modified-id="dependencies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>dependencies</a></span></li><li><span><a href="#read-input" data-toc-modified-id="read-input-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>read input</a></span></li><li><span><a href="#part-1" data-toc-modified-id="part-1-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>part 1</a></span><ul class="toc-item"><li><span><a href="#answer" data-toc-modified-id="answer-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>answer</a></span></li></ul></li><li><span><a href="#part-2" data-toc-modified-id="part-2-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>part 2</a></span><ul class="toc-item"><li><span><a href="#answer" data-toc-modified-id="answer-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>answer</a></span></li></ul></li></ul></div>

## dependencies

In [1]:
using Combinatorics, DataStructures, OffsetArrays, Interact, Images, NBInclude

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-15582617280321233273\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-12260769949214023566\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-15582617280321233273\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

## read input

In [57]:
function parse_range(line)
    m = match(r"(\d+)-(\d+)", line)
    return (l = parse(Int, m[1]), u = parse(Int, m[2]))
end

parse_range (generic function with 1 method)

In [40]:
function parse_field(line)
    key, value_s = split(line, ": ")
    values = parse_range.(split(value_s, " or "))
    
    return (; key, values)
end

parse_field (generic function with 1 method)

In [17]:
function parse_ticket(s)
    return parse.(Int, split(s, ','))
end

parse_ticket (generic function with 1 method)

In [37]:
function parse_input(filename)
    fields_s, mine_s, others_s = split.(split(read(filename, String), "\n\n"), '\n')
    
    fields = parse_field.(fields_s)
    mine = parse_ticket(mine_s[2])
    others = parse_ticket.(others_s[2:end])
    
    return (; fields, mine, others)
end

parse_input (generic function with 1 method)

In [58]:
input_sample_1 = parse_input("input_sample_1.txt")

(fields = NamedTuple{(:key, :values),Tuple{SubString{String},Array{NamedTuple{(:l, :u),Tuple{Int64,Int64}},1}}}[(key = "class", values = [(l = 1, u = 3), (l = 5, u = 7)]), (key = "row", values = [(l = 6, u = 11), (l = 33, u = 44)]), (key = "seat", values = [(l = 13, u = 40), (l = 45, u = 50)])], mine = [7, 1, 14], others = [[7, 3, 47], [40, 4, 50], [55, 2, 20], [38, 6, 12]])

In [4]:
input_sample_2 = parse_input("input_sample_2.txt")

String[]

In [59]:
input_puzzle = parse_input("input_puzzle.txt")

(fields = NamedTuple{(:key, :values),Tuple{SubString{String},Array{NamedTuple{(:l, :u),Tuple{Int64,Int64}},1}}}[(key = "departure location", values = [(l = 25, u = 80), (l = 90, u = 961)]), (key = "departure station", values = [(l = 41, u = 133), (l = 148, u = 968)]), (key = "departure platform", values = [(l = 48, u = 425), (l = 451, u = 952)]), (key = "departure track", values = [(l = 25, u = 371), (l = 384, u = 966)]), (key = "departure date", values = [(l = 49, u = 531), (l = 546, u = 973)]), (key = "departure time", values = [(l = 45, u = 641), (l = 656, u = 954)]), (key = "arrival location", values = [(l = 43, u = 357), (l = 364, u = 969)]), (key = "arrival station", values = [(l = 40, u = 669), (l = 689, u = 954)]), (key = "arrival platform", values = [(l = 40, u = 550), (l = 570, u = 956)]), (key = "arrival track", values = [(l = 49, u = 854), (l = 863, u = 953)]), (key = "class", values = [(l = 48, u = 601), (l = 614, u = 964)]), (key = "duration", values = [(l = 27, u = 698),

## part 1

In [67]:
function in_range(x, field)
    for (l, u) in field.values
        (x ∈ l:u) && (return true) 
    end
    return false
end

in_range (generic function with 1 method)

In [80]:
function invalid_values(tickets, fields)
    result = Int[]
    for ticket in tickets, v in ticket
        !any(in_range.(v, fields)) && push!(result, v)
    end
    
    return result
end

invalid_values (generic function with 1 method)

### answer

In [82]:
function show_answer_report(input, ::Val{:part1})
    @show vs = invalid_values(input.others, input.fields)
    @info "Answer found." answer=sum(vs)
    return
end

show_answer_report (generic function with 1 method)

In [83]:
show_answer_report(input_sample_1, Val(:part1))

vs = invalid_values(input.others, input.fields) = [4, 55, 12]


┌ Info: Answer found.
│   answer = 71
└ @ Main In[82]:3


In [ ]:
show_answer_report(input_sample_2, Val(:part1))

In [84]:
show_answer_report(input_puzzle, Val(:part1))

vs = invalid_values(input.others, input.fields) = [24, 8, 5, 987, 992, 4, 9, 978, 991, 9, 13, 992, 998, 18, 7, 984, 20, 6, 20, 8, 986, 980, 13, 22, 981, 985, 9, 9, 996, 985, 9, 982, 978, 989, 987, 987, 980, 18, 990, 983, 998, 0, 975, 15, 996, 999, 981, 9, 996, 987]


┌ Info: Answer found.
│   answer = 27898
└ @ Main In[82]:3


## part 2

In [87]:
function valid_ticket(ticket, fields)
    for v in ticket
        any(in_range.(v, fields)) || return false
    end
    return true
end

valid_ticket (generic function with 1 method)

In [85]:
function filter_valid_tickets(tickets, fields)
    return filter(t -> valid_ticket(t, fields), tickets)
end

filter_valid_tickets (generic function with 1 method)

In [105]:
function get_all_descriptions(tickets, fields)
    tickets = filter_valid_tickets(tickets, fields)
    
    result = []
    for i in 1:length(tickets[1])
        @show get_description(gather_entries(tickets, i), fields)
    end
end

get_all_descriptions (generic function with 1 method)

In [94]:
function gather_entries(tickets, n)
    return [t[n] for t in tickets]
end

gather_entries (generic function with 1 method)

In [111]:
function get_all_valid_description(entries, fields)
    result = []
    for f in fields
        all(in_range.(entries, Ref(f))) && push!(result, f.key)
    end
    return result
end

get_all_valid_description (generic function with 1 method)

In [158]:
function f()
    descriptions = []
    for i in 1:length(input_puzzle.fields)
        choices = get_all_valid_description(gather_entries(tickets, i), input_puzzle.fields)
        push!(
            descriptions,
            (
                len = length(choices),
                i,
                choices,
            )
        )
    end
    
    sort!(descriptions, by = d -> d.len)
    
    actual_descriptions = Dict()
    for k in 1:length(descriptions)
        d = only(setdiff(descriptions[k].choices, values(actual_descriptions)))
        actual_descriptions[descriptions[k].i] = d
    end
    
    result = []
    for (k, v) in actual_descriptions
        if contains(v, "departure")
            push!(result, k)
        end
    end
    return result
end

f (generic function with 1 method)

In [159]:
f()

6-element Array{Any,1}:
  2
 16
 17
  3
 13
 18

In [164]:
prod(input_puzzle.mine[f()])

2766491048287

### answer

In [106]:
function show_answer_report(input, ::Val{:part2})
    @show vs = get_all_descriptions(input.others, input.fields)
    @info "Answer found." answer=sum(vs)
    return
end

show_answer_report (generic function with 2 methods)

In [107]:
show_answer_report(input_sample_1, Val(:part2))

get_description(gather_entries(tickets, i), fields) = "class"
get_description(gather_entries(tickets, i), fields) = "class"
get_description(gather_entries(tickets, i), fields) = "seat"
vs = get_all_descriptions(input.others, input.fields) = nothing


LoadError: MethodError: no method matching iterate(::Nothing)
Closest candidates are:
  iterate(!Matched::Pkg.Resolve.NodePerm, !Matched::Any...) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Pkg/src/Resolve/maxsum.jl:228
  iterate(!Matched::Test.GenericString) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Test/src/Test.jl:1589
  iterate(!Matched::Test.GenericString, !Matched::Integer) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Test/src/Test.jl:1589
  ...

In [ ]:
show_answer_report(input_sample_2, Val(:part2))

In [108]:
show_answer_report(input_puzzle, Val(:part2))

get_description(gather_entries(tickets, i), fields) = "departure location"
get_description(gather_entries(tickets, i), fields) = "departure location"
get_description(gather_entries(tickets, i), fields) = "departure location"
get_description(gather_entries(tickets, i), fields) = "arrival platform"
get_description(gather_entries(tickets, i), fields) = "arrival platform"
get_description(gather_entries(tickets, i), fields) = "arrival platform"
get_description(gather_entries(tickets, i), fields) = "row"
get_description(gather_entries(tickets, i), fields) = "arrival platform"
get_description(gather_entries(tickets, i), fields) = "departure location"
get_description(gather_entries(tickets, i), fields) = "arrival location"
get_description(gather_entries(tickets, i), fields) = "departure location"
get_description(gather_entries(tickets, i), fields) = "arrival location"
get_description(gather_entries(tickets, i), fields) = "departure location"
get_description(gather_entries(tickets, i), fields) 

LoadError: MethodError: no method matching iterate(::Nothing)
Closest candidates are:
  iterate(!Matched::Pkg.Resolve.NodePerm, !Matched::Any...) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Pkg/src/Resolve/maxsum.jl:228
  iterate(!Matched::Test.GenericString) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Test/src/Test.jl:1589
  iterate(!Matched::Test.GenericString, !Matched::Integer) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Test/src/Test.jl:1589
  ...